# Builds 2sec sound files as new wavs

- generates 2-s mel spectrograms based on concatenated class wavs
#####  Notes :
- Runs in ROI_generation

In [1]:
import os
import matplotlib
matplotlib.use('Agg') # No pictures displayed 
import pylab
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import joblib
import math

%matplotlib inline

C:\Users\cq73\anaconda3\envs\ROI_generation\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\cq73\anaconda3\envs\ROI_generation\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [3]:
# where the melspecs will be generated. Pathway needs to exist, next chunk will create class folders IN the folder
mel_spec_dir = "../results/melspecs/"

# melspec dir from previous script
wav_dir = '../data/wavs_2sec/'

# where class metadata lives
cutoff_class_name = joblib.load('../data/cutoff_class_name_demo.pkl')
cutoff_class_name

roi_len = 2.000 # IMPORTANT for mfcc duration

In [4]:
# build a mel spec directory for each class
for i in range(len(cutoff_class_name)):
    if(os.path.exists(os.path.join(mel_spec_dir, cutoff_class_name[i][1]))):
        print(cutoff_class_name[i][1], "directory already exists. Skipping...")
    else:
        os.mkdir(os.path.join(mel_spec_dir, cutoff_class_name[i][1]))

print("Created melspec dirs:")
print(os.listdir(mel_spec_dir))
print(len(os.listdir(mel_spec_dir)))

Created melspec dirs:
['AMA', 'AMU', 'AVT', 'BBI', 'OQU']
5


In [6]:
SR = 22050  # Sampling frequency
FEATURE = 'mel'  # feature representation

FMIN = 0 #250 # 1000
en = 0

n_sam = int(22050 * roi_len)
print(wav_dir)

../data/wavs_2sec/


In [14]:
# generate mel specs for each concatenated wav
for wav_nm in os.listdir(wav_dir):
    print(wav_nm)
    cnt = 0
    en+=1
    print('>>',en,'/20')

    y, sr = librosa.load(os.path.join(wav_dir, wav_nm), sr = 22050)        
    tot_sam = int(np.shape(y)[0]/n_sam)

    print('Total number of possible ROIs:', tot_sam)
    for n in range(tot_sam):

        y_21k = y[n*n_sam:(n+1)*n_sam]

        fig = plt.figure(1, frameon=False)
        fig.set_size_inches(6, 6)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)

        S = librosa.feature.melspectrogram(y=y_21k, 
                                           sr=22050, 
                                           n_mels=128,
                                           fmin = 0,                                     
                                           fmax=11025, 
                                           n_fft=728, 
                                           hop_length=32, 
                                           #win_length = None, 
                                           htk = True)

        librosa.display.specshow(librosa.power_to_db(S ** 1, ref=np.max), fmin=0, y_axis='linear')# , cmap = 'gray')

        directory = os.path.join(mel_spec_dir, wav_nm.split('.')[0] +'/' + (wav_nm.split('.')[0] + '_' +str(cnt) +'.png'))  # 'test.png'
        fig.savefig(directory)
        fig.clear()
        ax.cla()
        plt.clf()
        plt.close('all')

        cnt +=1

        print('Number of ROIs converted here:', cnt)
        print('-'*70)

AMA.wav
>> 2 /20
Total number of possible ROIs: 1
Number of ROIs converted here: 1
----------------------------------------------------------------------
AMU.wav
>> 3 /20
Total number of possible ROIs: 1
Number of ROIs converted here: 1
----------------------------------------------------------------------
AVT.wav
>> 4 /20
Total number of possible ROIs: 2
Number of ROIs converted here: 1
----------------------------------------------------------------------
Number of ROIs converted here: 2
----------------------------------------------------------------------
BBI.wav
>> 5 /20
Total number of possible ROIs: 1
Number of ROIs converted here: 1
----------------------------------------------------------------------
OQU.wav
>> 6 /20
Total number of possible ROIs: 1
Number of ROIs converted here: 1
----------------------------------------------------------------------


In [15]:
# Count the total no. of ROIs generated 
print(os.listdir(mel_spec_dir))
cn = 0

for i in range(len(os.listdir(mel_spec_dir))):
    print(os.listdir(mel_spec_dir)[i])
    print(len(os.listdir(os.path.join(mel_spec_dir, os.listdir(mel_spec_dir)[i]))))
    
    cn += len(os.listdir(os.path.join(mel_spec_dir, os.listdir(mel_spec_dir)[i])))
    
    print('-'*50)
print('TOTAL ROIs:', cn)

['AMA', 'AMU', 'AVT', 'BBI', 'OQU']
AMA
1
--------------------------------------------------
AMU
1
--------------------------------------------------
AVT
2
--------------------------------------------------
BBI
1
--------------------------------------------------
OQU
1
--------------------------------------------------
TOTAL ROIs: 6
